# Catchment delineation
This Notebook defines a function catchment_delineation for delineating the catchment area based on flow direction and outlet coordinates. It also provides an interactive plot for visualizing the catchment delineation results.

In [1]:
import numpy as np # Library for numerical operations
import plotly.express as px # Library for plotting
import matplotlib.pyplot as plt  # Library for plotting
from matplotlib import cm  # Colormap module for color representations
import rasterio # Library for reading and writing raster data
from ipywidgets import widgets, interact # Interactive widgets for Jupyter notebooks
import os

## Function for catchment delineation using flow direction
The function below delineates the catchment area by recursively tracing upstream from the outlet cell based on flow direction until it reaches the boundary of the catchment. It effectively identifies all cells contributing to the flow of water towards the outlet cell, thus delineating the catchment area.

In [2]:
def catchment_delineation(flow_direction, outlet_row, outlet_col):
    """
    Delineates the catchment area based on flow direction and the location of the catchment outlet.

    Args:
        flow_direction (numpy.ndarray): 2D array representing flow direction values.
        outlet_row (int): Row index of the catchment outlet cell.
        outlet_col (int): Column index of the catchment outlet cell.

    Returns:
        catchment_mask (numpy.ndarray): Binary mask indicating the delineated catchment area.
    """
    
    ######################
    ### Initialization ###
    ######################
    # Initialize a mask (boolean array) to track visited cells during reverse tracing
    visited = np.zeros_like(flow_direction, dtype=bool)

    # Initialize the catchment mask (boolean array) to represent the catchment area
    catchment_mask = np.zeros_like(flow_direction, dtype=bool)

    #######################
    ### Reverse tracing ###
    #######################   
    # Start reverse tracing from the outlet cell and explores neighboring cells recursively
    
    # Create the variable stack to keep track of the cells that need to be explored during the reverse tracing process.
    stack = [(outlet_row, outlet_col)] # Add the first catchment cell (outlet) to the stack
    
    while stack: # It continues this process while there are cells (in stack) to explore.
        
        current_row, current_col = stack.pop() # return and remove the last item from the stack

        # Check if the current cell has been visited
        if visited[current_row, current_col]:
            continue

        # Mark the current cell as visited
        visited[current_row, current_col] = True

        # Mark the current cell as part of the catchment
        catchment_mask[current_row, current_col] = True
        
        ##########################
        ### Neighbor Selection ###
        ##########################

        # Define a 3x3 array to represent the directions of the valid neighboring cells (connected to the current cell)
        neighbours_dir = np.array([(2,    4,  8),
                                   (1,    0, 16),
                                   (128, 64, 32)])

        # Iterate over neighboring cells in a 3x3 grid centered around the current cell
        
        neighboring_cells = [] # Initialize the variable to add valid neighboring cells
        
        for dr in [-1, 0, 1]: # change in the row index (-1: move up, 0: stay, 1: move down)
            for dc in [-1, 0, 1]: # change in the column index (-1: move left, 0: stay, 1: move right)
                
                if dr == 0 and dc == 0: # exclude the current cell itself
                    continue
                    
                new_row, new_col = current_row + dr, current_col + dc
                
                # Determine valid neighboring cells: check if each neighboring cell satisfies certain conditions
                if (0 <= new_row < flow_direction.shape[0] and # It's within the bounds of the array.
                    0 <= new_col < flow_direction.shape[1] and # It's within the bounds of the array.
                    not visited[new_row, new_col] and # It hasn't been visited yet.
                    flow_direction[new_row, new_col] == neighbours_dir[1+dr, 1+dc]):# Its flow direction matches neighbours_dir.         
                    
                    neighboring_cells.append((new_row, new_col)) # Valid neighboring cells are added
        
        #############################
        ### Recursive Exploration ###
        #############################
        # Add all the valid neighboring cells to the stack for further exploration
        stack.extend(neighboring_cells)
        
    # Return the catchment_mask, a boolean array indicating the delineated catchment are
    return catchment_mask

In [3]:
# List available flow direction files (raster maps files in ascii format)
files = [f for f in os.listdir('.') if os.path.isfile(f)]
fd_files = []
for s in files:
    if 'asc' in s and 'fd' in s and 'D8' in s:
        fd_files.append(s)
input_file_fd = widgets.Select(
    options=fd_files,
    description='flow direction files:',
    disabled=False
)
display(input_file_fd)

Select(description='flow direction files:', options=('fd_D8.asc', 'fd_D8_syn.asc'), value='fd_D8.asc')

In [4]:
# Open the selected raster map file
with rasterio.open(input_file_fd.value) as src:
    # Read the raster data as a numpy array
    fd = src.read(1)  # Read the first band (index 0)

    # Get metadata of the raster map
    fd_metadata = src.meta

In [5]:
# List available flow accumulation files (raster maps files in ascii format)
files = [f for f in os.listdir('.') if os.path.isfile(f)]
fa_files = []
for s in files:
    if 'asc' in s and 'fa' in s and 'D8' in s:
        fa_files.append(s)
input_file_fa = widgets.Select(
    options=fa_files,
    description='flow accumulation files:',
    disabled=False
)
display(input_file_fa)

Select(description='flow accumulation files:', options=('fa_D8.asc', 'fa_D8_syn.asc'), value='fa_D8.asc')

In [6]:
# Open the selected raster map file
with rasterio.open(input_file_fa.value) as src:
    # Read the raster data as a numpy array
    fa = src.read(1)  # Read the first band (index 0)

    # Get metadata of the raster map
    fa_metadata = src.meta

In [7]:
# List available dem files (raster maps files in ascii format)
files = [f for f in os.listdir('.') if os.path.isfile(f)]
dem_files = []
for s in files:
    if 'asc' in s and 'dem' in s:
        dem_files.append(s)
input_file_dem = widgets.Select(
    options=dem_files,
    description='dem files:',
    disabled=False
)
display(input_file_dem)

Select(description='dem files:', options=('dem.asc', 'dem_fill.asc', 'dem_syn.asc', 'dem_syn_fill.asc', 'dem_s…

In [8]:
# Open the selected raster map file
with rasterio.open(input_file_dem.value) as src:
    # Read the raster data as a numpy array
    dem = src.read(1)  # Read the first band (index 0)

    # Get metadata of the raster map
    dem_metadata = src.meta

## Interactive plotting function for catchment delineation

In [9]:
def plot3D_surf(z,elev,azim):

    # Create meshgrid for x and y coordinates
    x, y = np.meshgrid(np.arange(z.shape[1]), np.arange(z.shape[0]))
    
    # Create figure
    fig = plt.figure(figsize=(10, 5))
    fig.suptitle('DEM')
    
    # First subplot: 2D plot
    ax = fig.add_subplot(1, 2, 1)
    mesh = ax.imshow(np.flip(z, 0), cmap=cm.coolwarm)
    ax.set_xlabel('X axis')
    ax.set_ylabel('Y axis')
    fig.colorbar(mesh)
    
    # Second subplot: 3D plot
    ax = fig.add_subplot(1, 2, 2, projection='3d')
    color = cm.coolwarm((z - np.nanmin(z)) / (np.nanmax(z) - np.nanmin(z)))
    surf = ax.plot_surface(x, y, z,
                           rstride=1,
                           cstride=1,
                           facecolors=color,
                           linewidth=0.,
                           antialiased=True)
    
    # Set view angle for 3D plot
    ax.view_init(elev=elev, azim=azim)
    ax.set_xlabel('X axis')
    ax.set_ylabel('Y axis')
    ax.set_zlabel('Elevation')
    #set_axes_equal(ax)

In [10]:
nl,nc = fd.shape

def iplot_catchment_delineation(outlet_row = 0, outlet_col=0):
    
    # Run the delineation function to obtain the catchment mask
    catchment_mask = catchment_delineation(fd,outlet_row, outlet_col)
    
    a = catchment_mask*1
    from scipy.ndimage import binary_erosion
    k = np.zeros((3,3),dtype=int); k[1] = 1; k[:,1] = 1
    out = a-binary_erosion(a,k)
    
    # Display the catchment mask
    fig, (ax1, ax2) = plt.subplots(figsize = (12,5), ncols=2)
    ax1.imshow(catchment_mask*fa)
    ax2.imshow(out)
    plt.show()
    
    elev = 30
    azim = 220
    plot3D_surf(catchment_mask*dem,elev,azim)

# Create interactive widgets (sliders) for selecting the outlet coordinates
interact(iplot_catchment_delineation,outlet_row= (0,nl-1,1),outlet_col = (0,nc-1,1))

interactive(children=(IntSlider(value=0, description='outlet_row', max=49), IntSlider(value=0, description='ou…

<function __main__.iplot_catchment_delineation(outlet_row=0, outlet_col=0)>